In [ ]:
models_path = 'Hasil Model'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import joblib

## Importing all models into environment

### Weather Forecast

- 4 models in models_forecast
- forecast_future to forecast: def forecast_future(df, models, feature_sets, days_ahead=7)
- json feature: feature_forecast

In [ ]:
import pickle
import os

# Correct directory to where .pkl models were saved
weather_models_dir = 'Hasil Model'
models_pkl = {}

# Ensure the directory exists
if not os.path.exists(weather_models_dir):
    print(f"Directory not found: {weather_models_dir}")
else:
    # Iterate through all files in the directory
    for filename in os.listdir(weather_models_dir):
        # Check for .pkl extension and specific filename prefix to load only our models
        if filename.endswith('.pkl') and filename.startswith('forecast_weather_xgboost_'):
            model_path = os.path.join(weather_models_dir, filename)
            model_name_without_ext = os.path.splitext(filename)[0] # e.g., 'forecast_weather_xgboost_rainfall_mm'

            # Load the model using pickle
            with open(model_path, 'rb') as f:
                loaded_model = pickle.load(f)
            models_pkl[filename] = loaded_model

print(f"Loaded {len(models_pkl)} models from {weather_models_dir}")

Loaded 4 models from /content/drive/MyDrive/Asah-Capstone Project!/Dataset Thingy/Modelling/Hasil Model


/tmp/ipython-input-1233791142.py:21: UserWarning: [10:48:47] WARNING: /workspace/src/collective/../data/../common/error_msg.h:83: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  loaded_model = pickle.load(f)


In [ ]:
# Load models used to forecast weather features

models_forecast = {}
for full_name, model_obj in models_pkl.items():
    # First, remove the .pkl extension
    name_without_ext = full_name.split('.')[0]
    # Then, remove the 'forecast_weather_xgboost_' prefix
    new_name = name_without_ext.replace('forecast_weather_xgboost_', '')
    models_forecast[new_name] = model_obj

print("Keys in models_pkl before renaming:", list(models_pkl.keys()))
print("Keys in models_forecast after renaming:", list(models_forecast.keys()))

Keys in models_pkl before renaming: ['forecast_weather_xgboost_rainfall_mm.pkl', 'forecast_weather_xgboost_temperature_c.pkl', 'forecast_weather_xgboost_humidity_pct.pkl', 'forecast_weather_xgboost_wind_speed_kmh.pkl']
Keys in models_forecast after renaming: ['rainfall_mm', 'temperature_c', 'humidity_pct', 'wind_speed_kmh']


In [ ]:
# funstion to create features
def create_features(df, target, lags=[1,3,7], rolls=[3,7]):
    df = df.copy()

    # time-based
    df["month"] = df["date"].dt.month
    df["week"] = df["date"].dt.isocalendar().week.astype(int)
    df["dayofyear"] = df["date"].dt.dayofyear

    # lags per mine
    for lag in lags:
        df[f"{target}_lag{lag}"] = df.groupby("mine_id")[target].shift(lag)

    # rolling averages per mine
    for r in rolls:
        df[f"{target}_roll{r}"] = df.groupby("mine_id")[target].shift(1).rolling(r).mean()

    return df

In [ ]:
# targets to forecast
targets = ["rainfall_mm", "temperature_c", "humidity_pct", "wind_speed_kmh"]

In [ ]:
# Function to forecast each target
def forecast_future(df, models, feature_sets, days_ahead=7):
    df_future = df.copy()
    last_date = df["date"].max()

    future_rows = []

    for i in range(days_ahead):
        next_date = last_date + pd.Timedelta(days=i+1)

        for mine in df["mine_id"].unique():
            row = {
                "date": next_date,
                "mine_id": mine,
                "mine_id_enc": le.transform([mine])[0],
                "month": next_date.month,
                "week": next_date.isocalendar().week,
                "dayofyear": next_date.timetuple().tm_yday
            }

            temp_df = pd.concat([df_future, pd.DataFrame([row])], ignore_index=True)

            for target in targets:
                temp_df = create_features(temp_df, target)

                # ambil baris terakhir untuk predict
                pred_input = temp_df.iloc[-1:][feature_sets[target]]

                # predict
                row[target] = models[target].predict(pred_input)[0]

            #df_future = pd.concat([df_future, pd.DataFrame(row)], ignore_index=True, axis = 1)
            future_rows.append(row)

    return pd.DataFrame(future_rows)

In [ ]:
# Import features for modelling
import json

with open(f"{models_path}/feature_forecast.json") as f:
    feature_forecast = json.load(f)

### Weather Classification

In [1]:
# Import Model for Weather Classification
import pickle
import os

model_dir = 'Hasil Model'


with open(os.path.join(model_dir, "xgb_weather_classification.pkl"), "rb") as f:
    model = pickle.load(f)

with open(os.path.join(model_dir, "feature_encoders.pkl"), "rb") as f:
    encoders = pickle.load(f)

with open(os.path.join(model_dir, "label_encoder_target.pkl"), "rb") as f:
    le_remark = pickle.load(f)



FileNotFoundError: [Errno 2] No such file or directory: 'Hasil Model/xgb_weather_classification.pkl'

### Model Effective Capacity: Forecast effective_capacity_ton_day

- model: efcap_model
- encoder: efcap_encoders

In [ ]:
# Load the encoders
efcap_encoders = joblib.load(f'{models_path}/encoders_efcap.pkl')

print(f"Label encoders loaded from: {models_path}/encoders_efcap.pkl")

Label encoders loaded from: /content/drive/MyDrive/Asah-Capstone Project!/Dataset Thingy/Modelling/Hasil Model/encoders_efcap.pkl


In [ ]:
# Load Model
with open(f'{models_path}/effcap_ranfor_regression.pkl', 'rb') as f:
    efcap_model = pickle.load(f)

### Model Production Plan: Forecast Actual production ton
- model = prodplan_model
- encoder = prodplan_encoders


In [ ]:
# Load the encoders
prodplan_encoders = joblib.load(f'{models_path}/encoders_prodplan.pkl')

print(f"Label encoders loaded from: {models_path}/encoders_prodplan.pkl")

Label encoders loaded from: /content/drive/MyDrive/Asah-Capstone Project!/Dataset Thingy/Modelling/Hasil Model/encoders_prodplan.pkl


In [2]:
# Load Model
prodplan_model = joblib.load(f'{models_path}/model_production_plan.pkl')

NameError: name 'joblib' is not defined

## Load Dataset (only for testing)


In [ ]:
test_dir = 'testing_dataset'

In [ ]:
weather_test = pd.read_csv(f'{test_dir}/weather_testing.csv')
efcap_test = pd.read_csv(f'{test_dir}/efcap_testing.csv')
prodplan_test = pd.read_csv(f'{test_dir}/prodplan_testing.csv')

In [ ]:
efcap_test.head()

,mine_id,equipment_type,road_condition,weather_condition,availability_pct
0,MINE_3,Excavator,Good,Mendung,75
1,MINE_1,Dump Truck,Good,Mendung,99
2,MINE_1,Excavator,Good,Mendung,98
3,MINE_4,Excavator,Good,Mendung,87
4,MINE_2,Loader,Good,Mendung,71


In [ ]:
prodplan_test.head()

,road_condition,weather_condition,availability_pct,effective_capacity_ton_day,planned_output_ton
0,1,4,75,256.09,77721
1,1,4,99,1519.87,64325
2,1,4,98,399.23,64325
3,1,4,87,218.14,59578
4,1,4,71,26.07,81761


## Predict Model

In [ ]:
# Encoding mine_id
le = efcap_encoders['mine_id']

In [ ]:
# Weather Forecast
weather_test['date'] = pd.to_datetime(weather_test['date'])
forecast_future(weather_test, models_forecast, feature_forecast, 7).head()

,date,mine_id,mine_id_enc,month,week,dayofyear,rainfall_mm,temperature_c,humidity_pct,wind_speed_kmh
0,2025-11-08,MINE_1,0,11,45,312,0.493276,25.312319,68.717224,0.207723
1,2025-11-08,MINE_2,1,11,45,312,0.411516,25.129068,68.466515,0.263520
2,2025-11-08,MINE_3,2,11,45,312,0.668059,25.844196,64.655190,0.223234
3,2025-11-08,MINE_4,3,11,45,312,0.564971,25.558550,68.584282,0.292800
4,2025-11-08,MINE_5,4,11,45,312,0.552145,25.665813,68.584282,0.284201


In [ ]:
# Effective Capacity
cat_cols_for_pred = efcap_test.select_dtypes(include=['object']).columns

for col in cat_cols_for_pred:
    if col in efcap_test.columns and col in efcap_encoders:
        # Use the loaded encoder to transform the column
        efcap_test[col] = efcap_encoders[col].transform(efcap_test[col])
    else:
        print(f"Warning: Column '{col}' not found in prediction DataFrame or encoder not loaded.")

print("Encoded DataFrame for prediction:")
efcap_test.head()

Encoded DataFrame for prediction:


,mine_id,equipment_type,road_condition,weather_condition,availability_pct
0,2,1,1,4,75
1,0,0,1,4,99
2,0,1,1,4,98
3,3,1,1,4,87
4,1,2,1,4,71


In [ ]:
efcap_model.predict(efcap_test)

array([ 168.33574167, 1578.13355048,  164.66896167,  217.10037333,
         50.837245  ,   59.93481357,   48.19158   ])

In [ ]:
prodplan_test

,road_condition,weather_condition,availability_pct,effective_capacity_ton_day,planned_output_ton
0,1,4,75,256.09,77721
1,1,4,99,1519.87,64325
2,1,4,98,399.23,64325
3,1,4,87,218.14,59578
4,1,4,71,26.07,81761
5,0,4,86,50.64,77721
6,1,4,74,63.66,84764


In [ ]:
prodplan_model.predict(prodplan_test)

array([61148.52, 49471.25, 49476.74, 45517.43, 58930.55, 56321.3 ,
       71182.41])

# Models+Agentic AI

### Testing All Model Availibility

In [ ]:
models_path = 'Hasil Model'

efcap_encoders = joblib.load(f'{models_path}/encoders_efcap.pkl')
le = efcap_encoders['mine_id']

test_dir = 'testing_dataset'
weather_test = pd.read_csv(f'{test_dir}/weather_testing.csv')
efcap_test = pd.read_csv(f'{test_dir}/efcap_testing.csv')
prodplan_test = pd.read_csv(f'{test_dir}/prodplan_testing.csv')

weather_test['date'] = pd.to_datetime(weather_test['date'])

In [ ]:
def forecast_future(df, models, feature_sets, days_ahead=7):
    df_future = df.copy()
    last_date = df["date"].max()

    future_rows = []

    for i in range(days_ahead):
        next_date = last_date + pd.Timedelta(days=i+1)

        for mine in df["mine_id"].unique():
            row = {
                "date": next_date,
                "mine_id": mine,
                "mine_id_enc": le.transform([mine])[0],
                "month": next_date.month,
                "week": next_date.isocalendar().week,
                "dayofyear": next_date.timetuple().tm_yday
            }

            temp_df = pd.concat([df_future, pd.DataFrame([row])], ignore_index=True)

            for target in targets:
                temp_df = create_features(temp_df, target)

                # ambil baris terakhir untuk predict
                pred_input = temp_df.iloc[-1:][feature_sets[target]]

                # predict
                row[target] = models[target].predict(pred_input)[0]

            #df_future = pd.concat([df_future, pd.DataFrame(row)], ignore_index=True, axis = 1)
            future_rows.append(row)

    return pd.DataFrame(future_rows)

In [ ]:
models_forecast = {}
for full_name, model_obj in models_pkl.items():
    # First, remove the .pkl extension
    name_without_ext = full_name.split('.')[0]
    # Then, remove the 'forecast_weather_xgboost_' prefix
    new_name = name_without_ext.replace('forecast_weather_xgboost_', '')
    models_forecast[new_name] = model_obj

print("Keys in models_pkl before renaming:", list(models_pkl.keys()))
print("Keys in models_forecast after renaming:", list(models_forecast.keys()))

Keys in models_pkl before renaming: ['forecast_weather_xgboost_rainfall_mm.pkl', 'forecast_weather_xgboost_temperature_c.pkl', 'forecast_weather_xgboost_humidity_pct.pkl', 'forecast_weather_xgboost_wind_speed_kmh.pkl']
Keys in models_forecast after renaming: ['rainfall_mm', 'temperature_c', 'humidity_pct', 'wind_speed_kmh']


In [ ]:
weather_pred = forecast_future(
    weather_test,
    models=models_forecast,
    feature_sets=feature_forecast,
    days_ahead=7
)

print("OK: weather_pred")

OK: weather_pred


In [ ]:
import joblib

weather_class_model = joblib.load(
    '/content/drive/MyDrive/Asah-Capstone Project!/Dataset Thingy/Modelling/Hasil Model/xgb_weather_classification.pkl'
)

In [ ]:
weather_class_input = weather_test.copy()
weather_class_input["date"] = weather_class_input["date"].astype("int64") / 1e9
weather_class_features_full = ["date", "rainfall_mm", "temperature_c", "wind_speed_kmh", "humidity_pct"]
weather_class = weather_class_model.predict(weather_class_input[weather_class_features_full])
print("OK: weather_class")

OK: weather_class


In [ ]:
prod_pred = prodplan_model.predict(prodplan_test)
print("OK: prod_pred")

OK: prod_pred


In [ ]:
ecap_cols = ["mine_id", "equipment_type", "road_condition", "weather_condition"]

for col in ecap_cols:
    efcap_test[col] = efcap_test[col].astype(str).str.strip()
    known = set(efcap_encoders[col].classes_)
    test_vals = set(efcap_test[col].unique())
    unknown = test_vals - known
    if unknown:
        print(f"Unseen values in {col}:", unknown)
    efcap_test = efcap_test[efcap_test[col].isin(efcap_encoders[col].classes_)]
    efcap_test[col] = efcap_encoders[col].transform(efcap_test[col])

efcap_pred = efcap_model.predict(efcap_test)
print("OK: efcap_pred")

OK: efcap_pred


### Input for Agentic AI

In [ ]:
import pandas as pd
import joblib
import json
import openai

In [ ]:
!pip install openai

In [ ]:
pip install --upgrade openai

In [ ]:
weather_class_categorical = le_remark.inverse_transform(weather_class)
print("Categorical Weather Classification:")
print(weather_class_categorical)

Categorical Weather Classification:
['Mendung' 'Mendung' 'Berawan' 'Mendung' 'Mendung' 'Mendung' 'Mendung'
 'Mendung' 'Mendung' 'Mendung' 'Mendung' 'Mendung' 'Mendung' 'Mendung'
 'Mendung' 'Mendung' 'Mendung' 'Mendung' 'Mendung' 'Mendung'
 'Hujan ringan' 'Mendung' 'Hujan ringan' 'Mendung' 'Mendung' 'Mendung'
 'Mendung' 'Mendung' 'Mendung' 'Mendung' 'Mendung' 'Hujan ringan'
 'Mendung' 'Mendung' 'Mendung']


In [ ]:
weather_output = pd.concat([weather_pred, pd.DataFrame(weather_class_categorical, columns=["weather_classification"])], axis=1)

In [ ]:
weather_output.head()

,date,mine_id,mine_id_enc,month,week,dayofyear,rainfall_mm,temperature_c,humidity_pct,wind_speed_kmh,weather_classification
0,2025-11-08,MINE_1,0,11,45,312,0.493276,25.312319,68.717224,0.207723,Mendung
1,2025-11-08,MINE_2,1,11,45,312,0.411516,25.129068,68.466515,0.263520,Mendung
2,2025-11-08,MINE_3,2,11,45,312,0.668059,25.844196,64.655190,0.223234,Berawan
3,2025-11-08,MINE_4,3,11,45,312,0.564971,25.558550,68.584282,0.292800,Mendung
4,2025-11-08,MINE_5,4,11,45,312,0.552145,25.665813,68.584282,0.284201,Mendung


In [ ]:
efcap_output = pd.concat([efcap_test, pd.DataFrame(efcap_pred, columns=["capacity_forecast"])], axis=1)

In [ ]:
prodplan_output = pd.concat([prodplan_test, pd.DataFrame(prod_pred, columns=["production_plan"])], axis=1)

In [ ]:
ml_output = {
    "weather_forecast": weather_output.to_dict(orient="records"),
    "capacity_forecast": efcap_output.to_dict(),
    "production_plan": prodplan_output.to_dict()
}

In [ ]:
def convert_timestamps(obj):
    if isinstance(obj, pd.Timestamp):
        return obj.strftime("%Y-%m-%d")
    if isinstance(obj, dict):
        return {k: convert_timestamps(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [convert_timestamps(i) for i in obj]
    return obj

In [ ]:
ml_output_clean = convert_timestamps(ml_output)

In [ ]:
json_schema = """
{
   "recommendations": [
      {
         "title": "",
         "action": "",
         "justification": "",
         "expected_impact": ""
      }
   ]
}
"""

### OpenAI as Agentic AI

In [ ]:
from openai import OpenAI
import json

client = OpenAI(api_key="sk-proj-OPENAI-API-KEY")

prompt = f"""
You are an expert mining operation optimization agent.

You will receive:
1. Weather forecast from an XGBoost model
2. Weather classification output
3. Effective capacity prediction
4. Actual production prediction
5. Known constraints (equipment, shipping, haulage, etc.)

Your task:
- Analyze all data holistically
- Generate a minimum of **5 actionable recommendations**
- Every recommendation MUST include:
  - **What to do**
  - **Why it matters (justification)**
  - **Expected impact (qualitative or quantitative)**
- Output format MUST be valid JSON with the structure:

JSON Schema:
{json_schema}


Be specific, realistic, and operationally useful.

Very important:
- DO NOT include explanations outside the JSON.
- DO NOT add comments, markdown, or introductory text.
- Only output valid JSON.
- Write all contents **inside the JSON** in **formal Indonesian**, optimized for mining engineers and operational planners.

Here is the data you must analyze:
{json.dumps(ml_output_clean, indent=2)}

"""

# 3. Panggil Agentic AI
response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {"role": "system", "content": "You are a mining optimization agent."},
        {"role": "user", "content": prompt}
    ]
)

print(response.choices[0].message.content)


{
  "recommendations": [
    {
      "title": "Optimalkan Penggunaan Alat Mekanis dengan Ketersediaan Tinggi",
      "action": "Jadwalkan operasi alat berat khususnya yang memiliki availability rate >85% secara prioritas pada lini produksi utama selama periode 8-14 November 2025.",
      "justification": "Dari data forecast, nilai availability alat di beberapa area mencapai 98-99%, yaitu pada unit-unit tertentu. Dengan memaksimalkan alat yang ketersediaannya tinggi, risiko kehilangan waktu akibat alat rusak dapat diminimalisir.",
      "expected_impact": "Peningkatan output aktual 5-10% terhadap rencana produksi harian, penurunan waktu non-produktif akibat alat menunggu perbaikan."
    },
    {
      "title": "Rotasi dan Penambahan Pemeliharaan Jalan Angkut",
      "action": "Laksanakan inspeksi dan perbaikan jalan angkut secara bergiliran terutama pada area dengan kondisi jalan <baik> (indikator road_condition = 0), dan berikan prioritas pada jalur-jalur pengangkutan utama.",
      "j

In [ ]:
output_ai_dir = '/content/drive/MyDrive/Asah-Capstone Project!/Dataset Thingy/Output JSON Rekomendasi AI'

In [ ]:
import json
import os

ai_response_content = response.choices[0].message.content

# Ensure the output directory exists
os.makedirs(output_ai_dir, exist_ok=True)

# Define the output file path
output_file_path = os.path.join(output_ai_dir, 'recommendations.json')

# Parse the content and save it as a JSON file
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(json.loads(ai_response_content), f, ensure_ascii=False, indent=2)

print(f"AI recommendations saved to: {output_file_path}")

AI recommendations saved to: /content/drive/MyDrive/Asah-Capstone Project!/Dataset Thingy/Output JSON Rekomendasi AI/recommendations.json
